In [ ]:
open IntegerFactorization;

operation Foo() : (Int, Int) {
    return FactorSemiprimeInteger(21);
}    

In [ ]:
%simulate Foo

In [ ]:
%workspace reload

In [21]:
%version

Component,Version
iqsharp,0.11.2003.3107
Jupyter Core,1.2.36563.0
.NET Runtime,".NETCoreApp,Version=v3.1"
